# WordNet Augmented Word Overlap

Based on [Saric2012](#Saric2012) design.

In [ ]:
from pandas import DataFrame
from nltk.corpus import wordnet
import numpy as np

In [1]:
s1 = 'John Smith makes boats in the United States of America'
s2 = 'John fabricates ships in US'

index = s2.split()
columns = s1.split()

m=[]
for wa in index:
    row = []
    syna = wordnet.synsets(wa)
    for wb in columns:
        synb = wordnet.synsets(wb)
        mx = [was.path_similarity(wbs) for was in syna for wbs in synb]
        mx = [item for item in mx if item is not None]
        if len(mx) > 0:
            row.append(max(mx))
        else:
            row.append(0.0)
    m.append(row)
M = np.asarray(m,dtype=np.float16)
df = DataFrame(M,index=index,columns=columns)

In [2]:
df.take([ix for ix in range(len(df.index))])

,John,Smith,makes,boats,in,the,United,States,of,America
John,1.000000,0.166626,0.090881,0.111084,0.083313,0,0.000000,0.111084,0,0.111084
fabricates,0.076904,0.083313,0.500000,0.111084,0.166626,0,0.166626,0.142822,0,0.099976
ships,0.090881,0.099976,0.250000,0.333252,0.250000,0,0.250000,0.199951,0,0.125000
in,0.083313,0.099976,0.071411,0.076904,1.000000,0,0.000000,0.333252,0,0.142822
US,0.083313,0.099976,0.066650,0.076904,0.333252,0,0.000000,0.333252,0,1.000000


#Similarity Ecuation
$p = {\sum_{w\in\ sent_1}max(df[w][w']) \over len(sent_1)} \ \ \ \forall\ w' \in\ sent_2$

$q = {\sum_{w'\in\ sent_2}max(df[w][w']) \over len(sent_2)} \ \ \ \forall\ w \in\ sent_1$

$sim = \left\{ \begin{array}{rcl} 
0  & if\ p+q = 0\\
{2 p*q \over (p+q)}  & others\\
\end{array}
\right.$

In [22]:
p=0
for wi in index:
    m = 0
    for wc in columns:
        m = max(m, df[wc][wi])
    p += m
p = p/len(index)

q=0
for wc in columns:
    m = 0
    for wi in index:
        m = max(m, df[wc][wi])
    q += m
q = q/len(columns)

sim = 2*p*q/(p+q or 1)
print(sim)

0.573675650301


<a id='references'></a>
## References

* <a id='Saric2012'></a>**[Saric2012]** Frane Šarić et all. *TakeLab: Systems for Measuring Semantic Text Similarity*.<br>
    In Proceedings of the First Joint Conference on Lexical and Computational Semantics (SEM '12).<br>
    Association for Computational Linguistics, 2012, 1, 441-448.